In [3]:
import pandas as pd
import requests

# 1. Leia o CSV corretamente (verifique separador e encoding)
df = pd.read_csv("corpo_docente_limpo.csv", delimiter=',', encoding='utf-8', engine='python')
print(df.columns)
print(df.head())

# 2. Filtre apenas quem tem ORCID preenchido
df = df[df['ORCID'].notnull() & (df['ORCID'] != '') & (df['ORCID'] != 'n/a')]

# 3. Função robusta para buscar dados detalhados do ORCID
def get_orcid_works(orcid_id):
    url = f"https://pub.orcid.org/v3.0/{orcid_id}/works"
    headers = {"Accept": "application/json"}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code != 200:
            return {
                "n_publicacoes": 0,
                "titulos": "",
                "anos": "",
                "tipos": "",
                "dois": ""
            }
        data = response.json()
        works = data.get("group", [])
        titulos, anos, tipos, dois = [], [], [], []
        for w in works:
            work_summary_list = w.get('work-summary', [])
            if not work_summary_list or work_summary_list[0] is None:
                # Se não existe summary válido, adiciona vazio e segue
                titulos.append("")
                anos.append("")
                tipos.append("")
                dois.append("")
                continue
            summary = work_summary_list[0]
            # Título da publicação
            titulo = ""
            try:
                titulo = summary.get('title', {}).get('title', {}).get('value', '')
            except Exception:
                pass
            titulos.append(titulo)
            # Ano de publicação - robusto para None
            pub_year = ""
            try:
                pub_date = summary.get('publication-date', None)
                if pub_date and isinstance(pub_date, dict):
                    year_val = pub_date.get('year', {})
                    pub_year = year_val.get('value', '') if year_val else ''
            except Exception:
                pass
            anos.append(pub_year)
            # Tipo do trabalho
            tipo = ""
            try:
                tipo = summary.get('type', '')
            except Exception:
                pass
            tipos.append(tipo)
            # DOI se disponível
            doi_val = ''
            try:
                ext_ids = summary.get('external-ids', {}).get('external-id', [])
                for ext in ext_ids:
                    if ext.get('external-id-type', '') == 'doi':
                        doi_val = ext.get('external-id-value', '')
                        break
            except Exception:
                pass
            dois.append(doi_val)
        return {
            "n_publicacoes": len(works),
            "titulos": "; ".join(titulos),
            "anos": "; ".join(anos),
            "tipos": "; ".join(tipos),
            "dois": "; ".join(dois)
        }
    except Exception as e:
        # Se der QUALQUER erro, retorna vazio e segue!
        return {
            "n_publicacoes": 0,
            "titulos": "",
            "anos": "",
            "tipos": "",
            "dois": ""
        }

# 4. Aplique a função e expanda os resultados em colunas
orcid_info = df['ORCID'].apply(get_orcid_works).apply(pd.Series)
df_final = pd.concat([df.reset_index(drop=True), orcid_info], axis=1)

# 5. Salve o resultado final
df_final.to_excel("corpo_docente_orcid_detalhado.xlsx", index=False)

print("Arquivo 'corpo_docente_orcid_detalhado.xlsx' gerado!")


Index(['Nome', 'Vínculo', 'Categoria', 'Regime', 'Grau/Título', 'ORCID'], dtype='object')
                                           Nome  \
0                   Alexandra Águeda Figueiredo   
1              Ana Cláudia Pires da Silva Pinto   
2               Ana Cristina Barata Pires Lopes   
3                  Ana Cristina Nata dos Santos   
4  Ana Isabel Loureiro de Sousa Virgolino Silva   

                                             Vínculo          Categoria  \
0  Contrato de Trabalho em Funções Públicas por t...  Professor Adjunto   
1  Contrato de Trabalho em Funções Públicas por t...  Professor Adjunto   
2  Contrato de Trabalho em Funções Públicas por t...  Professor Adjunto   
3  Contrato de Trabalho em Funções Públicas por t...  Professor Adjunto   
4  Contrato de Trabalho em Funções Públicas por t...  Professor Adjunto   

          Regime Grau/Título                ORCID  
0  Exclusividade     Doutora  0000-0003-3354-0727  
1  Exclusividade     Doutora  0000-0002-0803-564